In [6]:
!pip install spacy
!python -m spacy download pt_core_news_sm

[+] Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


2022-12-06 19:07:06.156860: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-12-06 19:07:06.156905: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-06 19:07:10.297432: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-12-06 19:07:10.297464: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-06 19:07:10.301541: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PA702MICRO100
2022-12-06 19:07:10.301714: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PA702MICRO100


In [7]:
import nltk
import spacy

import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer



In [8]:
# carrega os Tweets classificados
dfTweetsAux = pd.read_csv('dados/tweets_classificados.txt')

In [9]:
# Carrega StopWords
nlp = spacy.load('pt_core_news_sm')
nltk.download('stopwords')

# stopwords NLTK
stops_nltk = nltk.corpus.stopwords.words('portuguese')

# stopwords SpaCy
stops_spacy = nlp.Defaults.stop_words

# stopwords do SpaCy e NLTK combinadas
stops = list(set(stops_spacy).union(set(stops_nltk)))
len(stops)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\logonpflocal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


500

In [10]:
# função de lematização completa do documento
def fn_lematiza_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        sent.append(word.lemma_)
        
    return ' '.join(sent)

# função de lematização dos verbos do documento
def fn_lematiza_verb_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        if word.pos_ =='VERB':
            sent.append(word.lemma_)
        else:
            sent.append(word.text)
    return " ".join(sent)


# função para limpar documento
def fn_limpa_texto(texto):
    texto = re.sub(r'@\w+','',texto)
    texto = re.sub(r'#','',texto)
    texto = re.sub(r'RT[\s]+','',texto)  
    #texto = re.sub(r'rt[\s]+','',texto)  
    texto = re.sub(r'https:/\/\S+','',texto)
    texto = re.sub(r'[,.:;]','',texto)
    texto = re.sub(r'\n',' ',texto)
    texto = re.sub(u'[^a-zA-ZâêîôûÂÊÎÔÛáéíóúÁÉÍÓÚàèìòùÀÈÌÒÙãõÃÕçÇ ]', '', texto)
    texto = texto.strip().lower()
    
    return texto


In [11]:
# aplica normalização no dataframe
dfTweetsAux['texto_trat'] = dfTweetsAux.texto.apply(fn_limpa_texto)

# aplica a lematização no dataframe
dfTweetsAux['texto_lemma'] = dfTweetsAux.texto_trat.apply(fn_lematiza_texto)

# aplica a lematização dos verbos no dataframe
dfTweetsAux['texto_lemma_verb'] = dfTweetsAux.texto_trat.apply(fn_lematiza_verb_texto)

# apaga tweets nulos
dfTweetsAux.dropna(inplace=True)

In [12]:
# analisa alguns dados
print(dfTweetsAux['texto'][1])
print(dfTweetsAux['texto_trat'][1])
print(dfTweetsAux['texto_lemma'][1])
print(dfTweetsAux['texto_lemma_verb'][1])

@sandroottoni_ Se ninguem entender deixa que a professora Julia ensina :)
se ninguem entender deixa que a professora julia ensina
se ninguir entender deixar que o professora julio ensina
se ninguir entender deixar que a professora julia ensina


In [ ]:
# Modelo final

# vetorização tf-idf com combinação de unigrama e bigrama
vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=True) 

# texto tratado e verbos lematizado
vetor.fit(dfTweetsAux.texto_lemma_verb)
text_vect = vetor.transform(dfTweetsAux.texto_lemma_verb)

# separa as amostras de Treino (80%) e Texte (20%)
X_train,X_test,y_train,y_test = train_test_split(
        text_vect, 
        dfTweetsAux['sentimento'], 
        test_size = 0.2,
        random_state = 42
    )

# define e treina o modelo de classificação
modelo = SVC(kernel="rbf", C=5, class_weight="balanced")

modelo.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo.predict(X_test)
accuracy = accuracy_score(y_prediction, y_test)

print(text_vect.shape)
print(accuracy)


In [ ]:
# salva modelo e vetor de bag of words
import pickle

pickle.dump(vetor, open('modelo_verorizacao.pkl', 'wb'))
pickle.dump(modelo, open('modelo_classificacao.pkl', 'wb'))